In [1]:
using Rocket
using Distributions
using ReactiveMP
using BenchmarkTools

import Base: show

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1278
┌ Warning: Package ReactiveMP does not have Rocket in its dependencies:
│ - If you have ReactiveMP checked out for development and have
│   added Rocket as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ReactiveMP
└ Loading Rocket into ReactiveMP from project dependency, future warnings for ReactiveMP are suppressed.


In [2]:
function createSubgraph(model, index::Int)
    noise_add = make_node(model, +)
    
    noise = constvar(model, :noise, NormalMeanVariance(0.0, 200.0))
    yn    = datavar(model, :yn, Dirac{Float64})
    
    connect!(noise_add, :in2, noise)
    connect!(noise_add, :out, yn)
    
    return (yn, noise_add)
end

function createGraph(size::Int)    
    model = Model(message_out_transformer = DefaultMessageOutTransformer() + AsyncStreamTransformer())
    
    c0_add = make_node(model, +)
    
    c0       = constvar(model, :c0, 1.0)
    x0_prior = datavar(model, :x0_prior, NormalMeanVariance{Float64})
    
    connect!(c0_add, :in1, x0_prior)
    connect!(c0_add, :in2, c0)
    
    index = 1
    
    prev_c_add = c0_add
    
    xs = Vector{RandomVariable}(undef, size)
    ys = Vector{DataVariable}(undef, size)
    
    while index < size
        yn, noise_add = createSubgraph(model, index)
        xn            = randomvar(model, :xn)
        
        c_add = make_node(model, +)
        cn    = constvar(model, :cn, 1.0)
        
        connect!(prev_c_add, :out, xn)
        connect!(noise_add, :in1, xn)
        connect!(c_add, :in1, xn)
        connect!(c_add, :in2, cn)
        
        activate!(model, prev_c_add)
        activate!(model, noise_add)
        
        xs[index] = xn
        ys[index] = yn
        
        prev_c_add = c_add
        
        index += 1
    end
    
    last_noise_add = make_node(model, +)
    
    x_last     = randomvar(model, :x_last)
    y_last     = datavar(model, :ylast, Dirac{Float64})
    last_noise = constvar(model, :last_noise, NormalMeanVariance(0.0, 200.0))
    
    connect!(prev_c_add, :out, x_last)
    connect!(last_noise_add, :in1, x_last)
    connect!(last_noise_add, :in2, last_noise)
    connect!(last_noise_add, :out, y_last)
    
    activate!(model)
    
    xs[size] = x_last
    ys[size] = y_last
    
    return (xs, ys, x0_prior)
end

createGraph (generic function with 1 method)

In [3]:
function smoothing(data)
    N = length(data)
    
    xs, ys, x_prior = createGraph(N);
    
    marginals     = Vector{NormalMeanVariance{Float64}}(undef, N)
    subscriptions = Vector{Teardown}(undef, N)
    
    for (index, x) in enumerate(xs)
        @inbounds subscriptions[index] = subscribe!(getmarginal(x), (d) -> marginals[index] = getdata(d))
        yield()
    end
    
    update!(x_prior, NormalMeanVariance(0.0, 10000.0))
    for i in 1:N
        @inbounds update!(ys[i], data[i])
    end
    
    for i in 1:N
        yield()
    end
    
    foreach(unsubscribe!, subscriptions)
    
    return marginals
end

smoothing (generic function with 1 method)

In [4]:
N = 100_000
data = collect(1:N) + sqrt(200.0) * randn(N);

In [5]:
@time smoothing(data); # Initial compilation

 39.807592 seconds (104.49 M allocations: 7.852 GiB, 34.87% gc time)


In [6]:
@time smoothing(data); # Subsequent runs

 36.539132 seconds (97.45 M allocations: 7.484 GiB, 38.87% gc time)


In [7]:
smoothing(data)

100000-element Array{NormalMeanVariance{Float64},1}:
 NormalMeanVariance{Float64}(μ=0.9360581571971125, v=0.001999999600000074)
 NormalMeanVariance{Float64}(μ=1.9360581571971127, v=0.001999999600000074)
 NormalMeanVariance{Float64}(μ=2.936058157197112, v=0.001999999600000074)
 NormalMeanVariance{Float64}(μ=3.9360581571971123, v=0.001999999600000074)
 NormalMeanVariance{Float64}(μ=4.936058157197113, v=0.001999999600000074)
 NormalMeanVariance{Float64}(μ=5.936058157197112, v=0.001999999600000074)
 NormalMeanVariance{Float64}(μ=6.936058157197112, v=0.001999999600000074)
 NormalMeanVariance{Float64}(μ=7.936058157197113, v=0.001999999600000074)
 NormalMeanVariance{Float64}(μ=8.936058157197115, v=0.001999999600000074)
 NormalMeanVariance{Float64}(μ=9.936058157197113, v=0.0019999996000000736)
 NormalMeanVariance{Float64}(μ=10.936058157197113, v=0.0019999996000000736)
 NormalMeanVariance{Float64}(μ=11.936058157197111, v=0.0019999996000000736)
 NormalMeanVariance{Float64}(μ=12.93605815719711, v